In [4]:
import numpy as np
from numpy.linalg import inv
import math
import time
import argparse
import yaml

#VIRTUAL CAMERA OF THE SPHERICAL WRAPPER||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

fax=419.55; fay=419.55; ua0=500; va0=500; H=7551.9      

Kc         =  np.array([[fax, 0, ua0],[0, fay, va0],[0,0,1]])        
inv_Kc     =  inv(Kc)
h_vir      =  H
fc1_kc     =  np.array([[200, 0, 200],[0, 200, 75],[0,0,1]])
vc_kc_inv  =  inv_Kc
#|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
#SUBFUNCTION 1 TO REMAP, PIXELS TO COORDINATES-------------------------------------------------------------------1
def pix2cor(px, h, kc_inv): 

    cor_1   =  np.matmul( (h* kc_inv),px )
    
    return cor_1
#----------------------------------------------------------------------------------------------------------------1
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#SUBFUNCTION 2 TO LOAD THE REFERENCE POINTS FROM THE .YAML CALIBRATION FILES-------------------------------------2
def import_landmarks(lm_path, h_vir, vc_kc_inv):
    
    calib_points = lm_path
    h_vir        = h_vir 
    vc_kc_inv    = vc_kc_inv

    with open(calib_points) as c:                              

        points_data = yaml.load(c, Loader=yaml.FullLoader)     

    fish_points = points_data.get("fish_undistorted_points")  
    virtual_points = points_data.get("virtual_points")         

    f_points_list   = fish_points.get("data") 
    v_points_list   = virtual_points.get("data") 

    x1_v=v_points_list[0]; y1_v=v_points_list[4]      
    x2_v=v_points_list[1]; y2_v=v_points_list[5]
    x3_v=v_points_list[2]; y3_v=v_points_list[6]
    x4_v=v_points_list[3]; y4_v=v_points_list[7]

    v_points  =  np.array([ [ x1_v, x2_v, x3_v, x4_v ] , [ y1_v, y2_v, y3_v, y4_v ], [1,1,1,1] ]  )   
    v_cor     =  np.array([ [], [], [] ])   

    for i in range(4):   

        points = v_points[0:,i]                                                
        points_vec = np.array([ [points[0]] , [points[1]], [points[2]] ]  )    
        cord   = pix2cor(points_vec, h_vir, vc_kc_inv)                         

        if i==0:               
            v_cor = cord

        elif i>0:             

            v_cor= np.append(v_cor,cord,1)    

    x1_f=f_points_list[0]; y1_f=f_points_list[4]        
    x2_f=f_points_list[1]; y2_f=f_points_list[5]
    x3_f=f_points_list[2]; y3_f=f_points_list[6]
    x4_f=f_points_list[3]; y4_f=f_points_list[7]

    f_points  =  np.array([ [ x1_f, x2_f, x3_f, x4_f ] , [ y1_f, y2_f, y3_f, y4_f ], [1,1,1,1] ]  )   

    return(v_points, v_cor, f_points)
#----------------------------------------------------------------------------------------------------------------2
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#SUBFUNCTION 3 TO CALCULATE THE ROTATION-TRANSLATION MATRIX------------------------------------------------------3
def get_rt_mtx(fc1_kc, v_cor, f_points):
    
    fc1_kc     =  fc1_kc    
    v_cor      =  v_cor 
    f_points   =  f_points

    fax_f = fc1_kc[0,0];   fay_f = fc1_kc[1,1];   ua0_f = fc1_kc[0,2];   va0_f = fc1_kc[1,2]    
    
    v11=v_cor[0,0]; v21=v_cor[1,0]; v31=v_cor[2,0]   
    v12=v_cor[0,1]; v22=v_cor[1,1]; v32=v_cor[2,1]   
    v13=v_cor[0,2]; v23=v_cor[1,2]; v33=v_cor[2,2]
    v14=v_cor[0,3]; v24=v_cor[1,3]; v34=v_cor[2,3]   

    a1 = f_points[0,0];     a3 = f_points[0,1];     a5 = f_points[0,2];     a7 = f_points[0,3]
    a2 = f_points[1,0];     a4 = f_points[1,1];     a6 = f_points[1,2];     a8 = f_points[1,3]

    ext =  np.array([ [fax_f*v11,fax_f*v21,fax_f*v31,0,0,0,(ua0_f-a1)*v11,(ua0_f-a1)*v21],  
                      [0,0,0,fay_f*v11,fay_f*v21,fay_f*v31,(va0_f-a2)*v11,(va0_f-a2)*v21],
                      [fax_f*v12,fax_f*v22,fax_f*v32,0,0,0,(ua0_f-a3)*v12,(ua0_f-a3)*v22],
                      [0,0,0,fay_f*v12,fay_f*v22,fay_f*v32,(va0_f-a4)*v12,(va0_f-a4)*v22],
                      [fax_f*v13,fax_f*v23,fax_f*v33,0,0,0,(ua0_f-a5)*v13,(ua0_f-a5)*v23],
                      [0,0,0,fay_f*v13,fay_f*v23,fay_f*v33,(va0_f-a6)*v13,(va0_f-a6)*v23],
                      [fax_f*v14,fax_f*v24,fax_f*v34,0,0,0,(ua0_f-a7)*v14,(ua0_f-a7)*v24],
                      [0,0,0,fay_f*v14,fay_f*v24,fay_f*v34,(va0_f-a8)*v14,(va0_f-a8)*v24]   ]) 

    ext_inv = inv(ext)      
    
    ext_mtx_p=np.array([[(a1-ua0_f)*v31],[(a2-va0_f)*v31],  [(a3-ua0_f)*v31],[(a4-va0_f)*v31],  [(a5-ua0_f)*v31],[(a6-va0_f)*v31], [(a7-ua0_f)*v31],[(a8-va0_f)*v31]])

    rt_pms=np.matmul(ext_inv, ext_mtx_p)  

    r11=rt_pms[0,0];   r12=rt_pms[1,0];   r13=rt_pms[2,0];   r21=rt_pms[3,0];   r22=rt_pms[4,0];   r23=rt_pms[5,0];   r31=rt_pms[6,0];   r32=rt_pms[7,0]  

    rt=np.array([[r11,r12,r13],[r21,r22,r23],[r31,r32,1]])

    rt_inv=inv(rt)   

    return(rt, rt_inv)
#----------------------------------------------------------------------------------------------------------------3
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#SUBFUNCTION 4 TO EXPORT THE RT MATRIX AND ITS INVERSE TO .YAML FILE---------------------------------------------4
def export_mtx(saving_path, rt, rt_inv):

    data2exp = { 'rt_matrix': {'rows': 3, 'cols': 3,'data': rt.tolist()} , 'rt_inverse_matrix': {'rows': 3, 'cols': 3,'data': rt_inv.tolist()}   }

    matrix = rt
    with open(saving_path, 'w') as f:
        yaml.dump(data2exp, f,sort_keys=False)
      
    print ('\n'+ 'CALIBRATION RESULTS SUCCESSFULLY STORED')
    #print('\n'+'---------------------')
#----------------------------------------------------------------------------------------------------------------4    
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////    
#OBTAIN RT MATRIXES|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

#FRONTAL CAMERA--------------------------------

front_lm_path               =  'yaml_config/bird_calib_points_front.yaml'
front_saving_path           =  'yaml_config/b_calib_out_front.yaml'
v_points, v_cor, f_points   =   import_landmarks(front_lm_path, h_vir, vc_kc_inv) 
rt, rt_inv                  =   get_rt_mtx(fc1_kc, v_cor, f_points) 
export_mtx(front_saving_path, rt, rt_inv)

#LEFT CAMERA--------------------------------

left_lm_path                = 'yaml_config/bird_calib_points_left.yaml'
left_saving_path            = 'yaml_config/b_calib_out_left.yaml'
v_points, v_cor, f_points   =  import_landmarks(left_lm_path, h_vir, vc_kc_inv) 
rt, rt_inv                  =  get_rt_mtx(fc1_kc, v_cor, f_points) 
export_mtx(left_saving_path, rt, rt_inv)

#RIGHT CAMERA--------------------------------

right_lm_path               = 'yaml_config/bird_calib_points_right.yaml'
right_saving_path           = 'yaml_config/b_calib_out_right.yaml'
v_points, v_cor, f_points   =  import_landmarks(right_lm_path, h_vir, vc_kc_inv) 
rt, rt_inv                  =  get_rt_mtx(fc1_kc, v_cor, f_points) 
export_mtx(right_saving_path, rt, rt_inv)

#REAR CAMERA--------------------------------

back_lm_path               =  'yaml_config/bird_calib_points_back.yaml'
back_saving_path           =  'yaml_config/b_calib_out_back.yaml'
v_points, v_cor, f_points  =   import_landmarks(back_lm_path, h_vir, vc_kc_inv) 
rt, rt_inv                 =   get_rt_mtx(fc1_kc, v_cor, f_points) 
export_mtx(back_saving_path, rt, rt_inv)


CALIBRATION RESULTS SUCCESSFULLY STORED

CALIBRATION RESULTS SUCCESSFULLY STORED

CALIBRATION RESULTS SUCCESSFULLY STORED

CALIBRATION RESULTS SUCCESSFULLY STORED
